In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/home/shubham/Documents/NLP/BBC/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/shubham/Documents/NLP/BBC/bbc-summary.ipynb
/home/shubham/Documents/NLP/BBC/BBC News Summary/Summaries/sport/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/Summaries/entertainment/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/Summaries/politics/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/Summaries/tech/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/Summaries/business/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/News Articles/sport/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/News Articles/entertainment/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/News Articles/politics/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/News Articles/tech/163.txt
/home/shubham/Documents/NLP/BBC/BBC News Summary/News Articles/business/163.txt
/home/shubham/Documents/NLP/BBC/bbc news summary/BBC News Summary/Summaries/sport/163.txt
/home/shubham/Documents/NLP/BBC/bbc news summary/BBC News Summary/Summaries/entertai

In [2]:
pwd

'/home/shubham/Documents/NLP/BBC'

In [3]:
from transformers import pipeline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [4]:
from tqdm.notebook import tqdm
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# ! pip3 install -U git+https://github.com/huggingface/transformers.git

In [6]:
# ! pip3 install -U git+https://github.com/huggingface/accelerate.git

In [7]:
# !pip3 install -U accelerate

In [8]:
# !pip3 install -U transformers

In [9]:
!ls

'bbc news summary'  'BBC News Summary'	 bbc-summary.ipynb


In [10]:
with open('bbc news summary/BBC News Summary/News Articles/business/024.txt') as f:
    lines = f.readlines()

In [11]:
!ls 'bbc news summary/BBC News Summary/News Articles'

business  entertainment  politics  sport  tech


In [12]:
!ls 'bbc news summary/BBC News Summary/Summaries'

business  entertainment  politics  sport  tech


In [13]:
articles_files = [ f"bbc news summary/BBC News Summary/News Articles/{fol}" for fol in os.listdir('bbc news summary/BBC News Summary/News Articles/')]
summary_files = [ f"bbc news summary/BBC News Summary/Summaries/{fol}" for fol in os.listdir('bbc news summary/BBC News Summary/Summaries/')]

In [14]:
articles = []
summaries = []

In [15]:
for files in tqdm(articles_files):
    for file in os.listdir(files):
        with open(f'{files}/{file}', encoding='latin-1') as f:
            lines = f.readlines()
            lines = [  line.replace("\n", "")  for line in lines if line.replace("\n", "") != "" ]
            article = " ".join(lines)
            articles.append(["/".join(f"{files}/{file}".split('/')[-2:]), article])

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
for files in tqdm(summary_files):
    for file in os.listdir(files):
        with open(f'{files}/{file}', encoding='latin-1') as f:
            lines = f.readlines()
            lines = [ line.replace("\n", "")  for line in lines if line.replace("\n", "") != "" ]
            summary = " ".join(lines)
            summaries.append(["/".join(f"{files}/{file}".split('/')[-2:]), summary])

  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
summaries_df = pd.DataFrame(summaries, columns=['id', 'summary'])

In [18]:
articles_df = pd.DataFrame(articles, columns=['id', 'text'])

In [19]:
final_df = articles_df.merge(summaries_df, on='id', how='inner')

In [20]:
final_df

,id,text,summary
0,sport/163.txt,Arsenal through on penalties Arsenal win 4-2 o...,Michael Tonge and Harley wasted chances for th...
1,sport/087.txt,GB quartet get cross country call Four British...,Yelling clinched the women's European cross co...
2,sport/078.txt,Butler strikes gold in Spain Britain's Kathy B...,Gelete Burka then crowned a great day for Ethi...
3,sport/480.txt,Edgy Agassi struggles past Dent Andre Agassi p...,'s route to the fourth round was made easy whe...
4,sport/330.txt,Campese berates whingeing England Former Austr...,Former Australian wing David Campese has told ...
...,...,...,...
2220,business/181.txt,India and Russia in energy talks India and Rus...,Russian media reported that India and Russia s...
2221,business/424.txt,EU-US seeking deal on air dispute The EU and U...,Both sides hope to reach a negotiated deal ove...
2222,business/009.txt,Ethiopia's crop production up 24% Ethiopia pro...,"In 2003, crop production totalled 11.49 millio..."
2223,business/059.txt,Worldcom ex-boss launches defence Lawyers defe...,"The prosecution's star witness, former WorldCo..."


In [21]:
ds = Dataset.from_pandas(final_df, split='train')

In [22]:
ds

Dataset({
    features: ['id', 'text', 'summary', '__index_level_0__'],
    num_rows: 2225
})

In [23]:
dataset_bbc = ds.train_test_split(train_size=0.8, shuffle=True)

In [24]:
dataset_bbc

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'summary', '__index_level_0__'],
        num_rows: 1780
    })
    test: Dataset({
        features: ['id', 'text', 'summary', '__index_level_0__'],
        num_rows: 445
    })
})

In [25]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    

In [26]:

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)

In [27]:
model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Lay

In [28]:
def convert_features(x):
    input_encodings = tokenizer(x['text'] , max_length = 1024, truncation = True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(x['summary'], max_length = 128, truncation = True)

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }    

In [29]:
dataset_bbc_new = dataset_bbc.map(convert_features, batched = True)

Map:   0%|          | 0/1780 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

In [30]:
# dataset_bbc_new['train'][0]

In [31]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [33]:
# Trainer?

In [34]:
os.environ["WANDB_MODE"] = "dryrun"

In [35]:
trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_bbc_new["train"], 
                  eval_dataset=dataset_bbc_new["test"])

In [36]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: summary, id, __index_level_0__, text. If summary, id, __index_level_0__, text are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/home/shubham/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1780
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 111


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 3.95 GiB total capacity; 3.43 GiB already allocated; 13.69 MiB free; 3.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [36]:
## Save model
model.save_pretrained("pegasus-samsum-model")

In [37]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [48]:
os.listdir('/kaggle/working/')

['.virtual_documents',
 'tokenizer',
 'pegasus-samsum-model',
 'wandb',
 '__notebook_source__.ipynb',
 'pegasus-samsum']

In [49]:
#Load
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer")

In [55]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_bbc_new["test"][10]["text"]

reference = dataset_bbc_new["test"][10]["summary"]


In [56]:
pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

## 
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)

Dialogue:
Robertson out to retain Euro lure Hearts manager John Robertson hopes a place in the knock-out stages of the Uefa Cup could help keep some of his out-of-contract players at the club. "It could help. If we get through and have another European tie it may encourage players to stay at least until the end of the season," he said. "If we manage to get through it shows how well the club's progressing. "They have to think whether they are going to get other clubs like that should they decide to move on." A win for Robertson's side against Ferencvaros would put them through to the last 32 if Basle fail to beat Feyenoord. "It's very much the player's prerogative but the fact that we've been playing European football for the last three or four years is obviously an incentive," added Robertson. "But we want players who want to play for the football club, who are committed and a run in Europe always helps a little bit." With the game being played at Murrayfield instead of Tynecastle beca

In [57]:
print("\nModel Summary: ")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])


Model Summary: 
Robertson hopes a place in the knock-out stages of the Uefa Cup could help keep some of his out-of-contract players at the club.A win for Robertson's side against Ferencvaros would put them through to the last 32 if Basle fail to beat Feyenoord.The game is being played at Murrayfield instead of Tynecastle because of Uefa regulations, Robertson sees both positive and negative aspects to the change of venue.But he added: "If Tynecastle could have hosted 30,000 it would have been fantastic but that's one of the benefits of Murrayfield - it allows us to bring even
